## Author: Jesús Salazar Gómez, Costa Rica
## Course: Applied Data Science Capstone, week 4 and 5 (project), Coursera

## LinkedIn: https://www.linkedin.com/in/jesagocr/
## GitHub: https://github.com/jesagocr

In [5]:
# to open the url
from urllib.request import urlopen

# to work with the api
import json
import requests
import pandas as pd

# to work with the geolocalization data
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
Libraries imported.


In [1]:
# to open the url
from urllib.request import urlopen

# to work with the api
import json
import requests
import pandas as pd

# to work with the geolocalization data
#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [2]:
source = requests.get("https://es.wikipedia.org/wiki/Anexo:Playas_de_Costa_Rica").text
soup = BeautifulSoup(source, 'html.parser')

In [3]:
# primer intento de extraer la info con las playas

playas = []

for grossbeach in soup.find_all('li'):

    texto = str(grossbeach.text)
    #print(texto)

    try:
        if texto[:3] == "Pla" or texto[:3] == "Bal":
            #print('yes')
            playas.append(texto)
        else:
            #no.append(texto)
            pass
    except IndexError:
        pass

In [4]:
def newLine():
    print('********* -*- ********* ')

In [5]:
print(playas)

['Playa Barra del Colorado', 'Playa Bonita', 'Playa Cahuita (Blanca)', 'Playa Chiquita', 'Playa Gandoca', 'Playa Manzanillo', 'Playa Moín', 'Playa Negra (Puerto Viejo)', 'Playa Negra (Cahuita)', 'Playa Pirikiki', 'Playa Portete', 'Playa Puerto Vargas', 'Playas Tortuguero', 'Playa Vizcaya', 'Playa Westfalia', 'Playa Arbolito', 'Playa Avellanas', 'Playa Azul (Jurel)', 'Playa Bahía Junquillal', 'Playa Barco Quebrado', 'Playa Barrigona', 'Playa Bejuco', 'Playa Blanca (Santa Elena)', 'Playa Blanca y Virador', 'Playa Bongo', 'Playa Brasilito', 'Playa Buena Vista', 'Playa Camaronal', 'Playa Carbón', 'Playa Carmen', 'Playa Carrillo', 'Playa Cóncava', 'Playa Conchal', 'Playa Coyote', 'Playa Coyotera', 'Playa Cuajiniquil', 'Playa Culebra', 'Playas del Coco', 'Playa Flamingo', 'Playa Garza', 'Playa Grande', 'Playa Gringo', 'Playa Guiones', 'Playa Hermosa', 'Playa Iguanita', 'Playa Islita y Corozalito', 'Playa Jobo', 'Playa Junquillal', 'Playa La Penca', 'Playa Lagarto', 'Playa Langosta', 'Playa M

In [6]:
#todas los nombres con P o B (Playa o Balneario)

print(len(playas))
newLine()

print(playas[-1])
newLine()

playas = playas[:-1]
print(playas[-1])
newLine()

print(len(playas))

137
********* -*- ********* 
Playas de Costa Rica
********* -*- ********* 
Playa Zancudo
********* -*- ********* 
136


In [15]:
# Segundo intento sin BeatifulSoup para extraer quienes tienen info y quienes no
url = urlopen("https://es.wikipedia.org/wiki/Anexo:Playas_de_Costa_Rica")

In [16]:
content = str(url.read(),encoding='utf-8')
print(type(content))

<class 'str'>


In [12]:
#content = str(url.readline(), encoding='utf-8')
#print(type(content))

<class 'str'>


In [17]:
print(content)

mosa</a></li>
<li><a href="/w/index.php?title=Playa_Herradura&amp;action=edit&amp;redlink=1" class="new" title="Playa Herradura (aún no redactado)">Playa Herradura</a></li>
<li><a href="/w/index.php?title=Playa_Isla_Damas&amp;action=edit&amp;redlink=1" class="new" title="Playa Isla Damas (aún no redactado)">Playa Isla Damas</a></li>
<li><a href="/wiki/Jac%C3%B3" title="Jacó">Playa Jacó</a></li>
<li><a href="/w/index.php?title=Playa_Llorona&amp;action=edit&amp;redlink=1" class="new" title="Playa Llorona (aún no redactado)">Playa Llorona</a></li>
<li><a href="/w/index.php?title=Playa_Madrigal&amp;action=edit&amp;redlink=1" class="new" title="Playa Madrigal (aún no redactado)">Playa Madrigal</a></li>
<li><a href="/w/index.php?title=Playa_Mata_de_Lim%C3%B3n&amp;action=edit&amp;redlink=1" class="new" title="Playa Mata de Limón (aún no redactado)">Playa Mata de Limón</a></li>
<li><a href="/w/index.php?title=Playa_Mantas&amp;action=edit&amp;redlink=1" class="new" title="Playa Mantas (aún no r

In [20]:
data = content.split('<li><a href="')

In [23]:
print(data[1]) # the list of beaches start at index 1

/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>



In [24]:
### need to understand the data
for d in data[1:12]:
    print(d)

/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>

/w/index.php?title=Playa_Bonita_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Bonita (Limón) (aún no redactado)">Playa Bonita</a></li>

/w/index.php?title=Playa_Cahuita_(Blanca)&amp;action=edit&amp;redlink=1" class="new" title="Playa Cahuita (Blanca) (aún no redactado)">Playa Cahuita (Blanca)</a></li>

/wiki/Playa_Chiquita" title="Playa Chiquita">Playa Chiquita</a></li>

/w/index.php?title=Punta_Cocles&amp;action=edit&amp;redlink=1" class="new" title="Punta Cocles (aún no redactado)">Punta Cocles</a></li>

/w/index.php?title=Playa_Gandoca&amp;action=edit&amp;redlink=1" class="new" title="Playa Gandoca (aún no redactado)">Playa Gandoca</a></li>

/w/index.php?title=Playa_Manzanillo_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Manzanillo (Limón) (aún no redactado)">

In [25]:
# better a dict with info about which places have info (I guess because are popular) and which not
playas2 = {}

for d in data[1:-1]:
    inicio = (d.find('">') + 2)
    final = (d.find('</'))
    name = d[inicio:final]

    # it seems the most popular beaches are complete but the others are "aún no redactado"
    ini = d.find('aún no')

    if ini != -1: # if "aún no redactado" = no info of the beach, it is possible is not a popular beach
        playas2[name] = False
    else:
        playas2[name] = True

In [26]:
# also this dict have more beaches with punta or puerto that I did not count before
playas2

{'Playa Barra del Colorado': True,
 'Playa Bonita': False,
 'Playa Cahuita (Blanca)': False,
 'Playa Chiquita': True,
 'Punta Cocles': False,
 'Playa Gandoca': False,
 'Playa Manzanillo': False,
 'Playa Moín': False,
 'Playa Negra (Puerto Viejo)': False,
 'Playa Negra (Cahuita)': True,
 'Playa Pirikiki': False,
 'Playa Portete': False,
 'Playa Puerto Vargas': False,
 'Playas Tortuguero': True,
 'Playa Vizcaya': False,
 'Playa Westfalia': False,
 'Puerto Viejo de Talamanca': True,
 'Punta Mona': True,
 'Punta Uva': False,
 'Playa Arbolito': False,
 'Playa Avellanas': False,
 'Playa Azul (Jurel)': False,
 'Playa Bahía Junquillal': False,
 'Playa Barco Quebrado': False,
 'Playa Barrigona': False,
 'Playa Bejuco': False,
 'Playa Blanca (Santa Elena)': False,
 'Playa Blanca y Virador': False,
 'Playa Bongo': False,
 'Playa Brasilito': False,
 'Playa Buena Vista': False,
 'Playa Camaronal': False,
 'Playa Carbón': False,
 'Playa Carmen': False,
 'Playa Carrillo': True,
 'Playa Cóncava': Fals

In [27]:
len(playas2.keys())

137

In [17]:
source = requests.get("https://www.govisitcostarica.co.cr/travelInfo/beaches/beaches-list.asp").text

In [18]:
soup2 = BeautifulSoup(source, 'html.parser')

In [22]:
for item in soup2.find_all('<div class="pageListContentRow">'): # I need this but I need more practice with BS, it seems it just 
    print(item.text)                                            # works with tags and not other text

In [23]:
url2 = urlopen("https://www.govisitcostarica.co.cr/travelInfo/beaches/beaches-list.asp")

In [24]:
contenido = str(url2.read(), encoding='utf-8')

### There are 3 categories of beaches, use it to separete the places

In [29]:
types = contenido.split('<div class="pageListContentRow">')

In [41]:
soup = BeautifulSoup(types[1], 'html.parser')
for d in soup.find_all('div'):
    print(d.text)

Surfing Beaches
Snorkeling Beach


In [43]:
soup = BeautifulSoup(types[2], 'html.parser')
for d in soup.find_all('div'):
    print(d.text)
    print(len(d.text))

Bahía Salinas CahuitaP.N. CahuitaRefugio CamaronalDominicalEl RobleEsterillos EsteEsterillos OesteGarzaJacóP.N. Marino las Baulas MalpaísManuel AntonioMarbellaMatapaloPavonesPlaya AvellanaPlaya Bahia BallenaPlaya BejucoPlaya CaletasPlaya CoyotePlaya Doña AnaPlaya GrandePlaya GuionesPlaya HermosaPlaya JunquillalPlaya LangostaPlaya NegraPlaya NosaraPlaya San MiguelPlaya ZancudoPuerto LimonPuerto SoleyPuerto Viejo de TalamancaQueposP. N. Santa RosaSanta TeresaTamarindo
470
Bahía Salinas 
14
Cahuita
7
P.N. Cahuita
12
Refugio Camaronal
17
Dominical
9
El Roble
8
Esterillos Este
15
Esterillos Oeste
16
Garza
5
Jacó
4
P.N. Marino las Baulas 
23
Malpaís
7
Manuel Antonio
14
Marbella
8
Matapalo
8
Pavones
7
Playa Avellana
14
Playa Bahia Ballena
19
Playa Bejuco
12
Playa Caletas
13
Playa Coyote
12
Playa Doña Ana
14
Playa Grande
12
Playa Guiones
13
Playa Hermosa
13
Playa Junquillal
16
Playa Langosta
14
Playa Negra
11
Playa Nosara
12
Playa San Miguel
16
Playa Zancudo
13
Puerto Limon
12
Puerto Soley
12


In [39]:
soup = BeautifulSoup(types[3], 'html.parser')
for d in soup.find_all('div'):
    print(d.text)

Swimming Beaches



In [40]:
soup = BeautifulSoup(types[4], 'html.parser')
for d in soup.find_all('div'):
    print(d.text)

Refugio Bahía JunquillalP.N. BallenaCabuyaCahuitaP.N. CahuitaIsla del CañoCoronadoRefugio CurúDrake BayEl RobleGarzaRefugio GolfitoReserva Golfo DulceHacienda PinillaIsla BallenaIslitaJacóLa PalmaP.N. Marino las Baulas MalpaísManuel AntonioP.N. Manuel AntonioManzanilloMarbellaMatapaloMontezumaGolfo de PapagayoPaqueraParisminaP. N. Piedras BlancasPlaya Bahia BallenaPlaya BejucoPlaya BrasilitoPlaya Carrillo
Refugio Bahía Junquillal
P.N. Ballena
Cabuya
Cahuita
P.N. Cahuita
Isla del Caño
Coronado
Refugio Curú
Drake Bay
El Roble
Garza
Refugio Golfito
Reserva Golfo Dulce
Hacienda Pinilla
Isla Ballena
Islita
Jacó
La Palma
P.N. Marino las Baulas 
Malpaís
Manuel Antonio
P.N. Manuel Antonio
Manzanillo
Marbella
Matapalo
Montezuma
Golfo de Papagayo
Paquera
Parismina
P. N. Piedras Blancas
Playa Bahia Ballena
Playa Bejuco
Playa Brasilito
Playa Carrillo
Playa CativoPlaya ChiquitaPlaya ConchalPlaya CoyotePlaya FlamingoPlaya GrandePlaya GuionesPlaya HermosaPlaya HerraduraPlaya ManzanilloPlaya NaranjoPl

In [44]:
dd = {}

In [54]:
de = {}

In [56]:
from collections import defaultdict

In [57]:
de = defaultdict(dict)

In [58]:
de['playa1']['style'] = 'surf'

In [59]:
de

defaultdict(dict, {'playa1': {'style': 'surf'}})

In [60]:
de['playa1']['popular'] = True

In [61]:
de

defaultdict(dict, {'playa1': {'style': 'surf', 'popular': True}})

In [75]:
#to practice let's find the location of the first place in the list

address = 'Playa Zancudo, Costa Rica'

geolocator = Nominatim(user_agent="cr_explorer")
location = geolocator.geocode(address)
print(location)
#latitude = location.latitude
#longitude = location.longitude
#print(type(latitude))
#li.append(latitude)
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

Playa Zancudo, Ferrocarril del Sur, Kilometro 7, Florida, Golfito, Cantón Golfito, 60701, Costa Rica


In [71]:
beaches = playas2.keys()
len(beaches)

137

In [75]:
#

In [77]:
lista_beaches = list(beaches)
lista_beaches[:10]

['Playa Barra del Colorado',
 'Playa Bonita',
 'Playa Cahuita (Blanca)',
 'Playa Chiquita',
 'Punta Cocles',
 'Playa Gandoca',
 'Playa Manzanillo',
 'Playa Moín',
 'Playa Negra (Puerto Viejo)',
 'Playa Negra (Cahuita)']

In [78]:
latitudes = []
longitudes = []

for beach in lista_beaches[:70]:
    geolocator = Nominatim(user_agent="cr_explorer_js")
    address = beach+", Costa Rica"
    print(address)
    #print(type(address))

    location = geolocator.geocode(address)

    if location == None:
        latitudes.append(None)
        longitudes.append(None)
        
    else:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)

Playa Barra del Colorado, Costa Rica
Playa Bonita, Costa Rica
Playa Cahuita (Blanca), Costa Rica
Playa Chiquita, Costa Rica
Punta Cocles, Costa Rica
Playa Gandoca, Costa Rica
Playa Manzanillo, Costa Rica
Playa Moín, Costa Rica
Playa Negra (Puerto Viejo), Costa Rica
Playa Negra (Cahuita), Costa Rica
Playa Pirikiki, Costa Rica
Playa Portete, Costa Rica
Playa Puerto Vargas, Costa Rica
Playas Tortuguero, Costa Rica
Playa Vizcaya, Costa Rica
Playa Westfalia, Costa Rica
Puerto Viejo de Talamanca, Costa Rica
Punta Mona, Costa Rica
Punta Uva, Costa Rica
Playa Arbolito, Costa Rica
Playa Avellanas, Costa Rica
Playa Azul (Jurel), Costa Rica
Playa Bahía Junquillal, Costa Rica
Playa Barco Quebrado, Costa Rica
Playa Barrigona, Costa Rica
Playa Bejuco, Costa Rica
Playa Blanca (Santa Elena), Costa Rica
Playa Blanca y Virador, Costa Rica
Playa Bongo, Costa Rica
Playa Brasilito, Costa Rica
Playa Buena Vista, Costa Rica
Playa Camaronal, Costa Rica
Playa Carbón, Costa Rica
Playa Carmen, Costa Rica
Playa C

In [80]:
latitudes.count(None)

12

In [81]:
longitudes.count(None)

12

In [82]:
for beach in lista_beaches[70:]:
    geolocator = Nominatim(user_agent="cr_explorer_js")
    address = beach+", Costa Rica"
    print(address)
    #print(type(address))

    location = geolocator.geocode(address)

    if location == None:
        latitudes.append(None)
        longitudes.append(None)
        
    else:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)

Playa Pietra, Costa Rica
Playa Pitahaya, Costa Rica
Playa Pleito y San Juanillo, Costa Rica
Playa Pochote (Papagayo), Costa Rica
Playa Potrero, Costa Rica
Playa Real y Roble, Costa Rica
Playa Rajada, Costa Rica
Playa Sámara, Costa Rica
Playa San Miguel, Costa Rica
Playa Santa Teresa, Costa Rica
Playa Surco de Piedra, Costa Rica
Playa Prieta (Papagayo), Costa Rica
Playa Pochote (Salinas), Costa Rica
Playa Potrero Grande, Costa Rica
Playa Puerto Soley, Costa Rica
Playa Tamarindo, Costa Rica
Playa Ventanas, Costa Rica
Balnerario Puntarenas, Costa Rica
Playa Agujas, Costa Rica
Playa Bahía Drake, Costa Rica
Playa Ballena, Costa Rica
Playa Barú, Costa Rica
Playa Blanca, Costa Rica
Playa Boca Vieja, Costa Rica
Playa Cabo Blanco, Costa Rica
Playa Cabuya, Costa Rica
Playa Cocalito, Costa Rica
Playa Cacao, Costa Rica
Playa Caldera, Costa Rica
Playa Carate, Costa Rica
Playa Curú, Costa Rica
Playa de Arco, Costa Rica
Playa de Doña Ana, Costa Rica
Playa Dominical, Costa Rica
Playa Dominicalito, Cos

In [83]:
print(len(latitudes))
print(latitudes.count(None))
print(longitudes.count(None))

137
24
24
